Họ tên: Nguyễn Trung Dũng

MSSV: 19120486

# HW4: Song song hóa Radix Sort

Với các GPU tương đối mới thì để biên dịch chỉ cần dùng câu lệnh: \
`nvcc tên-file.cu -o tên-file-chạy`

Nhưng trên Colab mình thường lấy được GPU khá cũ là Tesla K80 với compute capability (phiên bản phần cứng) là 3.7; để biên dịch đúng với GPU khá cũ này thì bạn cần dùng câu lệnh: \
`nvcc -arch=sm_37 tên-file.cu -o tên-file-chạy` \
Trong đó, 37 chính là compute capability của GPU Tesla K80.

Để phòng trường hợp khi làm bài bạn lấy được GPU có compute capability x.x nhưng khi chấm bài Thầy lại lấy được GPU có compute capability khác x.x, dưới đây mình sẽ có đoạn code Python để tự động lấy 2 con số ứng với compute capability của GPU và lưu vào 2 biến `major` và `minor`:


In [1]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


Một khi đã chạy đoạn code Python ở trên, để biên dịch thì bạn sẽ dùng câu lệnh: \
`nvcc -arch=sm_{major}{minor} tên-file.cu -o tên-file-chạy`

Dưới đây, khi làm bài thì bạn có thể tùy ý thêm/xóa cell. Đừng xóa mấy cell có chữ của Thầy là được.

In [2]:
!nvcc -arch=sm_{major}{minor} HW4.cu -o HW4

In [3]:
!./HW4 256

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 9519.367 ms

Radix Sort by device
Time: 1016.128 ms
CORRECT :)


In [4]:
!./HW4 512

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 9448.010 ms

Radix Sort by device
Time: 617.838 ms
CORRECT :)


In [5]:
!./HW4 1024

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 9405.045 ms

Radix Sort by device
Time: 416.426 ms
CORRECT :)


Có thể thấy trong hàm `scanKernel`, trong giai đoạn scan cục bộ thì các block thực hiện sau cần phải chờ các block trước thực hiện xong để lấy kết quả `bSums[bi - 1]` nên việc tính mảng `bSums` coi như là thực thi tuần tự giữa các block (block `bi - 1` thực thi rồi tới block `bi`)</br>
Ta thấy với block size nhỏ hơn thì số lượng block thực thi lệnh lớn hơn; càng nhiều block chạy thì càng nhiều block phải chờ các block trước thực thi, làm tăng thời gian tính tuần tự mảng, tăng thời gian chạy chương trình